In [1]:
# imports
import numpy as np
import pandas as pd

In [24]:
# cleaning the data
need_to_clean = True
if need_to_clean:
    fnames = ['closeWilshire500', 'closeRussell1000']
    for fname in fnames:
        # read in data
        df = pd.read_csv(f'./data/{fname}.csv', index_col=0)
        # drop columns with no data
        df.dropna(axis=1, how='all', inplace=True)
        # format column names
        df.columns = [x.strip('.Close') for x in df.columns]
        # write clean data
        df.to_csv(f'./data/{fname}_clean.csv', index=False)
        df = None

In [ ]:
#